In [5]:
import os
import requests
import polars as pl
import pandas as pd
from bs4 import BeautifulSoup
races = pd.read_csv("data_raw/ergast/races.csv")

In [178]:
pl.Config(tbl_rows=50)

In [7]:
races = pl.from_pandas(races)
races

raceId,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str
1,2009,1,1,"""Australian Grand Prix""","""2009-03-29""","""06:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
2,2009,2,2,"""Malaysian Grand Prix""","""2009-04-05""","""09:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
3,2009,3,17,"""Chinese Grand Prix""","""2009-04-19""","""07:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
4,2009,4,3,"""Bahrain Grand Prix""","""2009-04-26""","""12:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
5,2009,5,4,"""Spanish Grand Prix""","""2009-05-10""","""12:00:00""","""http://en.wikipedia.org/wiki/2…","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N""","""\N"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
1140,2024,20,32,"""Mexico City Grand Prix""","""2024-10-27""","""20:00:00""","""https://en.wikipedia.org/wiki/…","""2024-10-25""","""18:30:00""","""2024-10-25""","""22:00:00""","""2024-10-26""","""17:30:00""","""2024-10-26""","""21:00:00""","""\N""","""\N"""
1141,2024,21,18,"""São Paulo Grand Prix""","""2024-11-03""","""17:00:00""","""https://en.wikipedia.org/wiki/…","""2024-11-01""","""14:30:00""","""2024-11-01""","""18:30:00""","""\N""","""\N""","""2024-11-02""","""18:00:00""","""2024-11-02""","""14:00:00"""
1142,2024,22,80,"""Las Vegas Grand Prix""","""2024-11-23""","""06:00:00""","""https://en.wikipedia.org/wiki/…","""2024-11-21""","""02:30:00""","""2024-11-21""","""06:00:00""","""2024-11-22""","""02:30:00""","""2024-11-22""","""06:00:00""","""\N""","""\N"""


In [11]:
races.with_columns(pl.col('url').map_elements(lambda x: x.split('/')[-1] + ".html").alias('filename')).select(pl.col(['raceId','filename']))

C:\Users\micha\AppData\Local\Temp\ipykernel_28660\2474267249.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  races.with_columns(pl.col('url').map_elements(lambda x: x.split('/')[-1] + ".html").alias('filename')).select(pl.col(['raceId','filename']))


raceId,filename
i64,str
1,"""2009_Australian_Grand_Prix.htm…"
2,"""2009_Malaysian_Grand_Prix.html"""
3,"""2009_Chinese_Grand_Prix.html"""
4,"""2009_Bahrain_Grand_Prix.html"""
5,"""2009_Spanish_Grand_Prix.html"""
…,…
1140,"""2024_Mexico_City_Grand_Prix.ht…"
1141,"""2024_S%C3%A3o_Paulo_Grand_Prix…"
1142,"""2024_Las_Vegas_Grand_Prix.html"""


In [13]:
dowl_loc = "data_raw/downloads/wikipedia/races"

In [86]:
def scrape_gp(source):
    with open(os.path.join(dowl_loc, source), 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file.read())
        gp = {"filename": source}
        table = soup.find(class_="infobox")
        rows = table.find_all("tr")
        data = ["Course","Course lenght","Distance","Weather","Date","Official name","Attendance"]
        for r in rows:
            for d in data:
                try:
                    if r.find("th").text == d:
                        gp[d] = r.find('td').text.strip()
                except:
                    pass
    return gp

In [172]:
def scrape_gp2(source):
    with open(os.path.join(dowl_loc, source), 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file.read(), 'html.parser')
        
        # Define the data keys we're looking for
        data_keys = ["Course", "Course length", "Distance", "Weather", "Date", "Official name", "Attendance"]
        
        # Create dictionary using a more efficient single pass through rows
        gp = {}
        for row in soup.find_all("tr"):
            th = row.find("th")
            if th and th.text in data_keys:
                td = row.find("td")
                if td:
                    gp[th.text] = td.text.strip()
                    
                    # Optional early exit when we've found all data
                    if len(gp) == len(data_keys):
                        break
                        
    return gp

In [82]:
%%timeit
scrape_gp("1988_German_Grand_Prix.html")

177 ms ± 1.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [176]:
scrape_gp("1988_German_Grand_Prix.html")

{'filename': '1988_German_Grand_Prix.html',
 'Date': '24 July 1988',
 'Official name': 'Mobil 1 Grosser Preis von Deutschland',
 'Course': 'Permanent racing facility',
 'Distance': '44 laps, 299.068 km (185.832 miles)',
 'Weather': 'Wet and cool'}

In [72]:
%%timeit
scrape_gp2("1988_German_Grand_Prix.html")

276 ms ± 3.44 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [90]:
gps = []
counter = 0
for file in os.listdir("data_raw/downloads/wikipedia/races"):
    counter += 1
    if counter % 50 == 0:
        print(f"{counter}: {file}")
    gps.append(scrape_gp(file))

50: 1956_Belgian_Grand_Prix.html
100: 1961_Italian_Grand_Prix.html
150: 1966_United_States_Grand_Prix.html
200: 1971_Canadian_Grand_Prix.html
250: 1974_United_States_Grand_Prix.html
300: 1978_Belgian_Grand_Prix.html
350: 1981_Dutch_Grand_Prix.html
400: 1984_Italian_Grand_Prix.html
450: 1987_Portuguese_Grand_Prix.html
500: 1990_United_States_Grand_Prix.html
550: 1994_Belgian_Grand_Prix.html
600: 1997_Austrian_Grand_Prix.html
650: 2000_Brazilian_Grand_Prix.html
700: 2003_Brazilian_Grand_Prix.html
750: 2005_United_States_Grand_Prix.html
800: 2008_Monaco_Grand_Prix.html
850: 2011_Indian_Grand_Prix.html
900: 2014_Austrian_Grand_Prix.html
950: 2016_Malaysian_Grand_Prix.html
1000: 2019_Austrian_Grand_Prix.html
1050: 2021_Russian_Grand_Prix.html
1100: 2023_United_States_Grand_Prix.html


In [91]:
gps

[{'filename': '1950_Belgian_Grand_Prix.html',
  'Date': '18 June 1950',
  'Official name': 'Grand Prix Automobile de Belgique',
  'Course': 'Permanent racing circuit',
  'Distance': '35 laps, 494.2 km (308.875 miles)',
  'Weather': 'Warm, dry and sunny'},
 {'filename': '1950_British_Grand_Prix.html',
  'Date': '13 May 1950',
  'Official name': "The Royal Automobile Club Grand Prix d'Europe Incorporating The British Grand Prix",
  'Course': 'Permanent racing facility',
  'Distance': '70 laps, 325.430\xa0km (202.213\xa0mi)',
  'Weather': 'Sunny, mild, dry.',
  'Attendance': '100,000[1]'},
 {'filename': '1950_French_Grand_Prix.html',
  'Date': '2 July 1950',
  'Official name': "XXXVII Grand Prix de l'A.C.F.",
  'Course': 'Temporary road course',
  'Distance': '64 laps, 500.160 km (310.785 miles)',
  'Weather': 'Hot and sunny'},
 {'filename': '1950_Indianapolis_500.html', 'Date': 'May 30, 1950'},
 {'filename': '1950_Italian_Grand_Prix.html',
  'Date': '3 September 1950',
  'Official name':

In [92]:
pl.DataFrame(gps)

filename,Date,Official name,Course,Distance,Weather,Attendance
str,str,str,str,str,str,str
"""1950_Belgian_Grand_Prix.html""","""18 June 1950""","""Grand Prix Automobile de Belgi…","""Permanent racing circuit""","""35 laps, 494.2 km (308.875 mil…","""Warm, dry and sunny""",null
"""1950_British_Grand_Prix.html""","""13 May 1950""","""The Royal Automobile Club Gran…","""Permanent racing facility""","""70 laps, 325.430 km (202.213 m…","""Sunny, mild, dry.""","""100,000[1]"""
"""1950_French_Grand_Prix.html""","""2 July 1950""","""XXXVII Grand Prix de l'A.C.F.""","""Temporary road course""","""64 laps, 500.160 km (310.785 m…","""Hot and sunny""",null
"""1950_Indianapolis_500.html""","""May 30, 1950""",null,null,null,null,null
"""1950_Italian_Grand_Prix.html""","""3 September 1950""","""XXI GRAN PREMIO D'ITALIA""","""Permanent racing facility""","""80 laps, 504.000 km (313.171 m…","""Warm, hot and sunny""",null
…,…,…,…,…,…,…
"""2024_Saudi_Arabian_Grand_Prix.…","""9 March 2024""","""Formula 1 STC Saudi Arabian Gr…","""Street Circuit""","""50 laps, 308.450 km (191.662 m…","""Clear""",null
"""2024_Singapore_Grand_Prix.html""","""22 September 2024""","""Formula 1 Singapore Airlines S…","""Temporary street circuit""","""62 laps, 306.143 km (190.228 m…","""Clear""","""269,072[2]"""
"""2024_Spanish_Grand_Prix.html""","""23 June 2024""","""Formula 1 Aramco Gran Premio d…","""Permanent racing facility""","""66 laps, 307.236 km (190.908 m…","""Clear""","""297,368[2]"""


In [98]:
pl.DataFrame(gps).write_json("data_raw/wiki_gps_raw.json")

In [182]:
pl.DataFrame(gps).filter(pl.col("Distance").is_null())

filename,Date,Official name,Course,Distance,Weather,Attendance
str,str,str,str,str,str,str
"""1950_Indianapolis_500.html""","""May 30, 1950""",null,null,null,null,null
"""1951_Indianapolis_500.html""","""May 30, 1951""",null,null,null,null,null
"""1952_Indianapolis_500.html""","""May 30, 1952""",null,null,null,null,null
"""1953_Indianapolis_500.html""","""May 30, 1953""",null,null,null,null,null
"""1954_Indianapolis_500.html""","""May 31, 1954""",null,null,null,null,null
"""1955_Indianapolis_500.html""","""May 30, 1955""",null,null,null,null,null
"""1956_Indianapolis_500.html""","""May 30, 1956""",null,null,null,null,null
"""1957_Indianapolis_500.html""","""May 30, 1957""",null,null,null,null,null
"""1958_Indianapolis_500.html""","""May 30, 1958""",null,null,null,null,null


In [180]:
pl.DataFrame(gps).filter(pl.col("Course").is_null())

filename,Date,Official name,Course,Distance,Weather,Attendance
str,str,str,str,str,str,str
"""1950_Indianapolis_500.html""","""May 30, 1950""",null,null,null,null,null
"""1951_Indianapolis_500.html""","""May 30, 1951""",null,null,null,null,null
"""1952_Indianapolis_500.html""","""May 30, 1952""",null,null,null,null,null
"""1953_Indianapolis_500.html""","""May 30, 1953""",null,null,null,null,null
"""1954_Indianapolis_500.html""","""May 31, 1954""",null,null,null,null,null
"""1955_Indianapolis_500.html""","""May 30, 1955""",null,null,null,null,null
"""1956_Indianapolis_500.html""","""May 30, 1956""",null,null,null,null,null
"""1957_Indianapolis_500.html""","""May 30, 1957""",null,null,null,null,null
"""1958_Indianapolis_500.html""","""May 30, 1958""",null,null,null,null,null


In [170]:
df.select(pl.col('Course')).unique()

Course
str
"""Temporary airport facility"""
"""Race track"""
"""Reims-Gueux"""
"""Temporary road circuit"""
"""Street circuit/Racing facility"""
…
"""Former Military Airbase"""
"""Permanent Road Facility"""
"""Yas Marina Circuit"""


In [184]:
pl.DataFrame(gps).filter(pl.col("Weather").is_null())

filename,Date,Official name,Course,Distance,Weather,Attendance
str,str,str,str,str,str,str
"""1950_Indianapolis_500.html""","""May 30, 1950""",null,null,null,null,null
"""1950_Monaco_Grand_Prix.html""","""21 May 1950""","""Prix de Monte-Carlo et XIe Gra…","""Street circuit""","""100 laps, 318.000 km (197.596 …",null,null
"""1951_Indianapolis_500.html""","""May 30, 1951""",null,null,null,null,null
"""1952_Indianapolis_500.html""","""May 30, 1952""",null,null,null,null,null
"""1953_Indianapolis_500.html""","""May 30, 1953""",null,null,null,null,null
"""1954_Indianapolis_500.html""","""May 31, 1954""",null,null,null,null,null
"""1955_Belgian_Grand_Prix.html""","""June 5, 1955""","""XVII Grote Prijs Van Belgie""","""Permanent racing facility""","""36 laps, 508.320 km (315.855 m…",null,null
"""1955_Indianapolis_500.html""","""May 30, 1955""",null,null,null,null,null
"""1955_Italian_Grand_Prix.html""","""September 11, 1955""","""XXVI Gran Premio d'Italia""","""Permanent road course""","""50 laps, 500.023 km (310.700 m…",null,null


In [102]:
pl.DataFrame(gps).filter(pl.col("Attendance").is_null())

filename,Date,Official name,Course,Distance,Weather,Attendance
str,str,str,str,str,str,str
"""1950_Belgian_Grand_Prix.html""","""18 June 1950""","""Grand Prix Automobile de Belgi…","""Permanent racing circuit""","""35 laps, 494.2 km (308.875 mil…","""Warm, dry and sunny""",null
"""1950_French_Grand_Prix.html""","""2 July 1950""","""XXXVII Grand Prix de l'A.C.F.""","""Temporary road course""","""64 laps, 500.160 km (310.785 m…","""Hot and sunny""",null
"""1950_Indianapolis_500.html""","""May 30, 1950""",null,null,null,null,null
"""1950_Italian_Grand_Prix.html""","""3 September 1950""","""XXI GRAN PREMIO D'ITALIA""","""Permanent racing facility""","""80 laps, 504.000 km (313.171 m…","""Warm, hot and sunny""",null
"""1950_Monaco_Grand_Prix.html""","""21 May 1950""","""Prix de Monte-Carlo et XIe Gra…","""Street circuit""","""100 laps, 318.000 km (197.596 …",null,null
…,…,…,…,…,…,…
"""2023_Azerbaijan_Grand_Prix.htm…","""30 April 2023 (2023-04-30)""","""Formula 1 Azerbaijan Grand Pri…","""Street circuit""","""51 laps, 306.049 km (190.170 m…","""Partly cloudy""",null
"""2023_Monaco_Grand_Prix.html""","""28 May 2023""","""Formula 1 Grand Prix de Monaco…","""Street circuit""","""78 laps, 260.286 km (161.772 m…","""Cloudy, with some rain interva…",null
"""2024_Monaco_Grand_Prix.html""","""26 May 2024""","""Formula 1 Grand Prix de Monaco…","""Street circuit""","""78 laps, 260.286 km (161.772 m…","""Sunny""",null


In [142]:
def dist(description):
    laps = float(description.split("laps")[0].strip())
    distance = float(description.split("laps, ")[1].split('km')[0].strip())
    return {'Laps':laps, 'Kms':distance}

In [166]:
df = races.with_columns(pl.col('url').map_elements(lambda x: x.split('/')[-1] + ".html").alias('filename')).select(pl.col(['raceId','filename'])).join(
    pl.DataFrame(gps).with_columns(
        pl.col("Distance").map_elements(dist).alias('x')
    ).unnest('x'), how='left', on='filename'
).select(pl.col(['raceId','Date','Official name','Course','Weather','Attendance','Laps','Kms'])).with_columns(
    pl.col('raceId').cast(float)
)

df

C:\Users\micha\AppData\Local\Temp\ipykernel_28660\746410434.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df = races.with_columns(pl.col('url').map_elements(lambda x: x.split('/')[-1] + ".html").alias('filename')).select(pl.col(['raceId','filename'])).join(
C:\Users\micha\AppData\Local\Temp\ipykernel_28660\746410434.py:2: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  pl.DataFrame(gps).with_columns(


raceId,Date,Official name,Course,Weather,Attendance,Laps,Kms
f64,str,str,str,str,str,f64,f64
1.0,"""29 March 2009""","""2009 Formula 1 ING Australian …","""Temporary street circuit""","""Sunny with temperatures reachi…",null,58.0,307.574
2.0,"""5 April 2009""","""2009 Formula 1 Petronas Malays…","""Permanent racing facility""","""Dry start, with heavy rain and…","""97,368 (Weekend)[2]""",31.0,171.833
3.0,"""19 April 2009""","""2009 Formula 1 Chinese Grand P…","""Permanent racing facility""","""Rain""","""120,000 (Weekend)[2]""",56.0,305.066
4.0,"""26 April 2009""","""2009 Formula 1 Gulf Air Bahrai…","""Permanent racing facility""","""Sunny""","""93,680 (Weekend)[2]""",57.0,308.238
5.0,"""10 May 2009""","""Formula 1 Gran Premio de Españ…","""Permanent racing facility""","""Warm, Sunny""","""92,500""",66.0,307.104
…,…,…,…,…,…,…,…
1140.0,"""27 October 2024""","""Formula 1 Gran Premio de la Ci…","""Permanent racing facility""","""Partly cloudy""","""404,958[2]""",71.0,305.354
1141.0,"""3 November 2024""","""Formula 1 Lenovo Grande Prêmio…","""Permanent racing facility""","""Rain""","""291,717[2]""",69.0,297.261
1142.0,"""November 23, 2024""","""Formula 1 Heineken Silver Las …","""Street circuit""","""Clear""","""306,000[3]""",50.0,309.958


In [168]:
df.write_parquet('data/gps_wikipedia.parquet')